## Sistema

In [ ]:
# Escolher uma FS melhor
# Simulação do sistema original:

# Amplitude do degrau de entrada.

amplitude = 1.0;

# Sistema de espaço de estados.

sys = ctrl.ss(A,B,C,D);

# Condições Iniciais.

x0 = np.matrix([[0], [0], [0]]);

# Simulação do sistema.

t = np.linspace(0, 100e-3, 1000);
u = amplitude*np.heaviside(t, 1);
Y, t, X = ctrl.matlab.lsim(sys, u, t, x0);

# Simulação com discretização dos integradores.
# I(z)=T/(z-1)=Xi(z)/Xi_ponto(z)=E(z)/E_ponto(z)
# xi(k)=T*xi_ponto(k-1)+xi(k-1)
# e(k)=T*e_ponto(k-1)+e(k-1)

T = 5e-4;
tfinal = 100e-3;
Kmax = int(tfinal/T+1);
k = np.linspace(0, Kmax, Kmax+1);
u = amplitude*np.heaviside(k, 1);

x1 = np.zeros_like(k);
x2 = np.zeros_like(k);
x3 = np.zeros_like(k);
x1_ponto = np.zeros_like(k);
x2_ponto = np.zeros_like(k);
x3_ponto = np.zeros_like(k);
y = np.zeros_like(k);

# Condições iniciais (podem ser diferentes de zero).

x1[0] = x0[0];  #  para k = 0
x2[0] = x0[1];  #  para k = 0
x3[0] = x0[2];  #  para k = 0
x1_ponto[0] = 0;  #  para k = 0
x2_ponto[0] = 0;  #  para k = 0
x3_ponto[0] = 0;  #  para k = 0
y[0] = C[0,0]*x1[0]+C[0,1]*x2[0]+C[0,2]*x3[0]+D[0,0]*u[0];  #  para k = 0

for j in range(1,Kmax+1):
    # Equações dos integradores:
    
    x1[j] = T*x1_ponto[j-1]+x1[j-1];
    x2[j] = T*x2_ponto[j-1]+x2[j-1];
    x3[j] = T*x3_ponto[j-1]+x3[j-1];    
    
    # Equação diferencial de estados:  Xponto=A*X+B*U
        
    x1_ponto[j] = A[0,0]*x1[j]+A[0,1]*x2[j]+A[0,2]*x3[j]+B[0,0]*u[j];  
    x2_ponto[j] = A[1,0]*x1[j]+A[1,1]*x2[j]+A[1,2]*x3[j]+B[1,0]*u[j];
    x3_ponto[j] = A[2,0]*x1[j]+A[2,1]*x2[j]+A[2,2]*x3[j]+B[2,0]*u[j]; 
    
    # Equação de Saída: Y=C*X+D*U
    
    y[j] = C[0,0]*x1[j]+C[0,1]*x2[j]+C[0,2]*x3[j]+D[0,0]*u[j];

# Comportamento da variável de estado x1(t).

plt.figure();
plt.plot(k*T, x1, ds='steps-post');
plt.plot(t, X.T[0], label='Simulação');
plt.plot(k*T, x1, 'r*', label='Equações recursivas');
plt.xlabel("kT");
plt.ylabel("$\mathrm{x_{1}(kT)}$");
plt.legend(loc='lower right');
plt.grid();
plt.tight_layout();
plt.show();

# Comportamento da variável de estado x2(t).

plt.figure();
plt.plot(k*T, x2, ds='steps-post');
plt.plot(t, X.T[1], label='Simulação');
plt.plot(k*T, x2, 'r*', label='Equações recursivas');
plt.xlabel("kT");
plt.ylabel("$\mathrm{x_{2}(kT)}$");
plt.legend(loc='lower right');
plt.grid();
plt.tight_layout();
plt.show();

# Comportamento da variável de estado x3(t).

plt.figure();
plt.plot(k*T, x3, ds='steps-post');
plt.plot(t, X.T[2], label='Simulação');
plt.plot(k*T, x3, 'r*', label='Equações recursivas');
plt.xlabel("kT");
plt.ylabel("$\mathrm{x_{3}(kT)}$");
plt.legend(loc='lower right');
plt.grid();
plt.tight_layout();
plt.show();

# Comportamento da saída y(t).

plt.figure();
plt.plot(k*T, y, ds='steps-post');
plt.plot(t, Y, label='Simulação');
plt.plot(k*T, y, 'r*', label='Equações recursivas');
plt.xlabel("kT");
plt.ylabel("$\mathrm{V_{o}(kT)}$");
plt.legend(loc='lower right');
plt.grid();
plt.tight_layout();
#plt.savefig('itemc.png', facecolor='w');
plt.show();

## Controlador servossistema tipo 0

In [ ]:
# Simulação do sistema controlado 'contínuo' e discreto com amplitude de 1,0 V para o sinal de entrada: 

# Amplitude do degrau de entrada.

amplitude = 1.0;

# Sistema expandido.

sistema_servo = ctrl.ss(AA,BB,CC,DD);

# Condições Iniciais.

x0 = np.matrix([[0], [0], [0]]);

# Simulação do sistema.

t = np.linspace(0, 120e-3, 1000);
R = amplitude*np.heaviside(t,1);
Y, t, X = ctrl.matlab.lsim(sistema_servo, R, t, x0);

X1a = X[:,0];
X1b = X[:,1];
X1c = X[:,2];

# Simulação com discretização dos integradores.
# I(z)=T/(z-1)=Xi(z)/Xi_ponto(z)=E(z)/E_ponto(z)
# xi(k)=T*xi_ponto(k-1)+xi(k-1)
# e(k)=T*e_ponto(k-1)+e(k-1)

T = 2e-4;
tfinal = 120e-3;
Kmax = int(tfinal/T+1);
k = np.linspace(0, Kmax, Kmax+1);
r = amplitude*np.heaviside(k, 1);

x1 = np.zeros_like(k);
x2 = np.zeros_like(k);
qsi = np.zeros_like(k);
u = np.zeros_like(k);
x1_ponto = np.zeros_like(k);
x2_ponto = np.zeros_like(k);
erro = np.zeros_like(k);
y = np.zeros_like(k);

# Condições iniciais (podem ser diferentes de zero).

x1[0] = x0[0];  #  para k = 0
x2[0] = x0[1];  #  para k = 0
qsi[0] = x0[2];  #  para k = 0
u[0] = -(K[0,0]*x1[0]+K[0,1]*x2[0])+ki*qsi[0];
x1_ponto[0] = 0;  #  para k = 0
x2_ponto[0] = 0;  #  para k = 0
y[0] = C[0,0]*x1[0]+C[0,1]*x2[0]+D*u[0];  #  para k = 0
erro[0] = r[0]-y[0];  #  para k = 0

for j in range(1,Kmax+1):
    # Equações dos integradores:
    
    x1[j] = T*x1_ponto[j-1]+x1[j-1];
    x2[j] = T*x2_ponto[j-1]+x2[j-1];    
    qsi[j] = T*erro[j-1]+qsi[j-1];
    
    # Equação da diferença do sinal de controle:
    
    u[j] = -(K[0,0]*x1[j]+K[0,1]*x2[j])+ki*qsi[j];
    
    # Equação diferencial de estados:  Xponto=A*X+B*U
    
    x1_ponto[j] = A[0,0]*x1[j]+A[0,1]*x2[j]+B[0]*u[j];  
    x2_ponto[j] = A[1,0]*x1[j]+A[1,1]*x2[j]+B[1]*u[j]; 
    
    # Equação de Saída: Y=C*X+D*U
    
    y[j] = C[0,0]*x1[j]+C[0,1]*x2[j]+D*u[j];
    
    # Equação diferencial do erro: Eponto=R-Y
    
    erro[j] = r[j]-y[j];
    
# Equação do sinal de controle 'contínuo':

U = np.zeros_like(t);
for j in range(len(t)):
    U[j] = -(K[0,0]*X1a[j]+K[0,1]*X1b[j])+ki*X1c[j];
    
# Equação do erro do sistema 'contínuo':    

E = np.zeros_like(t);
for j in range(len(t)):
    E[j] = R[j] - Y[j];
    
    
# Comportamento do estado x1:
plt.figure();
plt.plot(k*T, x1, ds='steps-post');
plt.plot(t, X1a, label='Simulação');
plt.plot(k*T, x1, 'r*', label='Equações recursivas');
plt.xlabel("t");
plt.ylabel("$\mathrm{x_{1}(t)}$");
plt.legend(loc='lower right');
plt.tight_layout();
plt.grid();
plt.show();

# Comportamento do estado x2:      
plt.figure();
plt.plot(k*T, x2, ds='steps-post');
plt.plot(t, X1b, label='Simulação');
plt.plot(k*T, x2, 'r*', label='Equações recursivas');
plt.xlabel("t");
plt.ylabel("$\mathrm{x_{2}(t)}$");
plt.legend(loc='lower right');
plt.tight_layout();
plt.grid();
plt.show();

# Comportamento da saída do erro:                       
plt.figure();
plt.plot(k*T, erro, ds='steps-post');
plt.plot(t, E, label='Simulação');
plt.plot(k*T, erro, 'r*', label='Equações recursivas');
plt.xlabel("t");
plt.ylabel("$\\dot{\\xi}$(t)");
plt.legend(loc='upper right');
plt.tight_layout();
plt.grid();
plt.show();

# Comportamento da ação de controle:                       
plt.figure();
plt.plot(k*T, u, ds='steps-post');
plt.plot(t, U, label='Simulação');
plt.plot(k*T, u, 'r*', label='Equações recursivas');
plt.xlabel("t");
plt.ylabel("u(t)");
plt.legend(loc='lower right');
plt.tight_layout();
plt.grid();
plt.show();

# Comportamento da saída y:
plt.figure();
plt.plot(k*T, y, ds='steps-post');
plt.plot(t, Y, label='Simulação');
plt.plot(k*T, y, 'r*', label='Equações recursivas');
plt.xlabel("t");
plt.ylabel("y(t)");
plt.legend(loc='lower right');
plt.tight_layout();
plt.grid();
plt.show();

## Controlador servossistema tipo 1

In [ ]:
K = np.matrix('25 5')

k1 = K[0, 0]

# simulação com discretização dos integradores
# I(z)=T/(z-1)=Xi(z)/Xi_ponto(z)
# xi(k)=T*xi_ponto(k-1)+xi(k-1)

T = 0.01;
tfinal = 5;
Kmax = int(tfinal/T+1);
k = np.linspace(0, Kmax, Kmax+1)
ref = np.ones(len(k));

x1 = np.zeros(len(k))
x2 = np.zeros(len(k))

x1_ponto = np.zeros(len(k))
x2_ponto = np.zeros(len(k))


y = C[0,0]*x1


erro = np.zeros(len(k))

u = np.zeros(len(k))
    for j in range(1,Kmax+1):
    # sistema original
        # Equações dos integradores
        x1[j]=T*x1_ponto[j-1]+x1[j-1];
        x2[j]=T*x2_ponto[j-1]+x2[j-1];    

        # equação de saída do observador de ordem mínima
        # Equação de Saída: Y=C*X+D*U
        y[j] = C[0,0]*x1[j]
        
        erro[j] = ref[j] - x1[j];
        
        u[j] = -(K[0,1]*x2[j]) + (k1*(erro[j]));
        # Equação diferencial de estados:  Xponto=A*X+B*U
        x1_ponto[j] = A[0,0]*x1[j]+A[0,1]*x2[j]+B[0]*u[j];  
        x2_ponto[j] = A[1,0]*x1[j]+A[1,1]*x2[j]+B[1]*u[j]; 


## Observador de ordem plena

In [ ]:
# Simulação do sistema com o observador de estados 'contínuo' e discreto com amplitude de 1,0 V para o sinal de entrada: 

# Amplitude do degrau de entrada.

amplitude = 1.0;

# Sistema expandido.

sys = ctrl.ss(AA,BB,CC,DD);

# Condições Iniciais.

x0 = np.matrix([[0], [0], [0], [0]]);

# Simulação do sistema expandido.

t = np.linspace(0, 120e-3, 1000);
U = amplitude*np.heaviside(t, 1);
Y, t, X = ctrl.matlab.lsim(sys, U, t, x0);

X1a = X[:,0];
X1b = X[:,1];
X1c = X[:,2];
X1d = X[:,3];

# Simulação com discretização dos integradores
# I(z)=T/(z-1)=Xi(z)/Xi_ponto(z)
# xi(k)=T*xi_ponto(k-1)+xi(k-1)

T = 2e-4;
tfinal = 120e-3;
Kmax = int(tfinal/T+1);
k = np.linspace(0, Kmax, Kmax+1);
u = amplitude*np.heaviside(k, 1);

x1 = np.zeros_like(k);
x2 = np.zeros_like(k);
x1e = np.zeros_like(k);
x2e = np.zeros_like(k);
x1_ponto = np.zeros_like(k);
x2_ponto = np.zeros_like(k);
x1e_ponto = np.zeros_like(k);
x2e_ponto = np.zeros_like(k);
y = np.zeros_like(k);
ye = np.zeros_like(k);

# Condições iniciais (podem ser diferentes de zero).

x1[0] = 0;  #  para k = 0
x2[0] = 0;  #  para k = 0
x1e[0] = 0;  #  para k = 0
x2e[0] = 0;  #  para k = 0
x1_ponto[0] = 0;  #  para k = 0
x2_ponto[0] = 0;  #  para k = 0
x1e_ponto[0] = 0;  #  para k = 0
x2e_ponto[0] = 0;  #  para k = 0
y[0] = C[0,0]*x1[0]+C[0,1]*x2[0]+D*u[0];  #  para k = 0
ye[0] = C[0,0]*x1e[0]+C[0,1]*x2e[0]+D*u[0];  #  para k = 0

for j in range(1,Kmax+1):
    
    # Equações dos integradores:
    
    x1[j] = T*x1_ponto[j-1]+x1[j-1];
    x2[j] = T*x2_ponto[j-1]+x2[j-1];
    x1e[j] = T*x1e_ponto[j-1]+x1e[j-1];
    x2e[j] = T*x2e_ponto[j-1]+x2e[j-1];
    
    # Equação diferencial de estados:  Xponto=A*X+B*U
    
    x1_ponto[j] = A[0,0]*x1[j]+A[0,1]*x2[j]+B[0]*u[j];  
    x2_ponto[j] = A[1,0]*x1[j]+A[1,1]*x2[j]+B[1]*u[j];
    
    # Equação de Saída: Y=C*X+D*U
    
    y[j] = C[0,0]*x1[j]+C[0,1]*x2[j]+D*u[j];
    ye[j] = C[0,0]*x1e[j]+C[0,1]*x2e[j]+D*u[j];
    
    # Equação diferencial de estados aproximados:  Xeponto=A*Xe+B*U+Ke*(Y-Ye)
    
    x1e_ponto[j] = A[0,0]*x1e[j]+A[0,1]*x2e[j]+B[0]*u[j]+Ke[0,0]*(y[j]-ye[j]);  
    x2e_ponto[j] = A[1,0]*x1e[j]+A[1,1]*x2e[j]+B[1]*u[j]+Ke[1,0]*(y[j]-ye[j]);
    
# Comportamento do estado x1:

plt.figure();
plt.plot(k*T, x1, ds='steps-post');
plt.plot(t, X1a, label='Simulação');
plt.plot(k*T, x1,'r*', label='Equações recursivas');
plt.xlabel("t");
plt.ylabel("$\mathrm{x_{1}(t)}$");
plt.legend(loc='lower right');
plt.tight_layout();
plt.grid();
plt.show();

# Comportamento do estado x2:

plt.figure();
plt.plot(k*T, x2, ds='steps-post');
plt.plot(t, X1b, label='Simulação');
plt.plot(k*T, x2, 'r*', label='Equações recursivas');
plt.xlabel("t");
plt.ylabel("$\mathrm{x_{2}(t)}$");
plt.legend(loc='lower right');
plt.tight_layout();
plt.grid();
plt.show();

# Comportamento da saída y:

plt.figure();
plt.plot(k*T, y, ds='steps-post');
plt.plot(t, Y.T[0], label='Simulação');
plt.plot(k*T, y, 'r*', label='Equações recursivas');
plt.xlabel("t");
plt.ylabel("y(t)");
plt.legend(loc='lower right');
plt.tight_layout();
plt.grid();
plt.show();

# Comportamento da estado x1e:

plt.figure();
plt.plot(k*T, x1e, ds='steps-post');
plt.plot(t, X1c, label='Simulação');
plt.plot(k*T, x1e, 'r*', label='Equações recursivas');
plt.xlabel("t");
plt.ylabel("$\mathrm{\\widetilde{x}_{1}(t)}$");
plt.legend(loc='lower right');
plt.tight_layout();
plt.grid();
plt.show();

# Comportamento da estado x2e:

plt.figure();
plt.plot(k*T, x2e, ds='steps-post');
plt.plot(t, X1d, label='Simulação');
plt.plot(k*T, x2e, 'r*', label='Equações recursivas');
plt.xlabel("t");
plt.ylabel("$\mathrm{\\widetilde{x}_{2}(t)}$");
plt.legend(loc='lower right');
plt.tight_layout();
plt.grid();
plt.show();

# Comportamento da saída ye:

plt.figure();
plt.plot(k*T, ye, ds='steps-post');
plt.plot(t, Y.T[2], label='Simulação');
plt.plot(k*T, ye, 'r*', label='Equações recursivas');
plt.xlabel("t");
plt.ylabel("$\mathrm{\\widetilde{y}(t)}$");
plt.legend(loc='lower right');
plt.tight_layout();
plt.grid();
plt.show();

## Observador de ordem mínima

In [ ]:
# x1e = x1till
# Simulação do sistema controlado 'contínuo' e discreto com amplitude de 1,0 V para o sinal de entrada: 

# Amplitude do degrau de entrada.

amplitude = 1.0;

# Sistema expandido.

sistema_min = ctrl.ss(AA,BB,CC,DD);

# Condições Iniciais.

x0 = np.matrix([[0], [0], [0], [0], [0]]);

# Simulação do sistema.

t = np.linspace(0, 100e-3, 1000);
U = amplitude*np.heaviside(t,1);
Y, t, X = ctrl.matlab.lsim(sistema_min, U, t, x0);

X1a = X[:,0];
X1b = X[:,1];
X1c = X[:,2];
X1d = X[:,3];
X1e = X[:,4];

# Simulação com discretização dos integradores.
# I(z)=T/(z-1)=Xi(z)/Xi_ponto(z)
# xi(k)=T*xi_ponto(k-1)+xi(k-1)

T = 5e-4;
tfinal = 100e-3;
Kmax = int(tfinal/T+1);
k = np.linspace(0, Kmax, Kmax+1);
u = amplitude*np.heaviside(k, 1);

x1 = np.zeros_like(k);
x2 = np.zeros_like(k);
x3 = np.zeros_like(k);
eta1 = np.zeros_like(k);
eta2 = np.zeros_like(k);
x1_ponto = np.zeros_like(k);
x2_ponto = np.zeros_like(k);
x3_ponto = np.zeros_like(k);
y = np.zeros_like(k);
eta1_ponto = np.zeros_like(k);
eta2_ponto = np.zeros_like(k);
x1e = np.zeros_like(k);
x2e = np.zeros_like(k);
x3e = np.zeros_like(k);

# Condições iniciais (podem ser diferentes de zero).

x1[0] = x0[0];  #  para k = 0
x2[0] = x0[1];  #  para k = 0
x3[0] = x0[2];  #  para k = 0
eta1[0] = x0[3];  #  para k = 0
eta2[0] = x0[4];  #  para k = 0
x1_ponto[0] = 0;  #  para k = 0
x2_ponto[0] = 0;  #  para k = 0
x3_ponto[0] = 0;  #  para k = 0
y[0] = C[0,0]*x1[0]+C[0,1]*x2[0]+C[0,2]*x3[0]+D[0,0]*u[0];  #  para k = 0
eta1_ponto[0] = x0[3];  #  para k = 0
eta2_ponto[0] = x0[4];  #  para k = 0
x1e[0] = 0; #  para k = 0
x2e[0] = 0; #  para k = 0
x3e[0] = 0; #  para k = 0

for j in range(1,Kmax+1):
    # Equações dos integradores:
    
    x1[j] = T*x1_ponto[j-1]+x1[j-1];
    x2[j] = T*x2_ponto[j-1]+x2[j-1];
    x3[j] = T*x3_ponto[j-1]+x3[j-1];
    eta1[j] = T*eta1_ponto[j-1]+eta1[j-1];
    eta2[j] = T*eta2_ponto[j-1]+eta2[j-1];    
    
    # Equação diferencial de estados:  Xponto=A*X+B*U
    
    x1_ponto[j] = A[0,0]*x1[j]+A[0,1]*x2[j]+A[0,2]*x3[j]+B[0,0]*u[j];
    x2_ponto[j] = A[1,0]*x1[j]+A[1,1]*x2[j]+A[1,2]*x3[j]+B[1,0]*u[j];
    x3_ponto[j] = A[2,0]*x1[j]+A[2,1]*x2[j]+A[2,2]*x3[j]+B[2,0]*u[j];
    
    # Equação de Saída: Y=C*X+D*U
    
    y[j] = C[0,0]*x1[j]+C[0,1]*x2[j]+C[0,2]*x3[j]+D[0,0]*u[j];
    
    # Equação diferencial de estados:  Nponto=A*N+B*Y+F*U
    
    eta1_ponto[j] = A_min[0,0]*eta1[j]+A_min[0,1]*eta2[j]+B_min[0,0]*y[j]+F_min[0,0]*u[j];
    eta2_ponto[j] = A_min[1,0]*eta1[j]+A_min[1,1]*eta2[j]+B_min[1,0]*y[j]+F_min[1,0]*u[j];
    
    # Equação de Saída: Xe=C*N+D*Y
    
    x1e[j] = C_min[0,0]*eta1[j]+C_min[0,1]*eta2[j]+D_min[0,0]*y[j];
    x2e[j] = C_min[1,0]*eta1[j]+C_min[1,1]*eta2[j]+D_min[1,0]*y[j];
    x3e[j] = C_min[2,0]*eta1[j]+C_min[2,1]*eta2[j]+D_min[2,0]*y[j];

# Comportamento do estado x1:

plt.figure();
plt.plot(k*T, x1, ds='steps-post');
plt.plot(t, X1a, label='Simulação');
plt.plot(k*T, x1,'r*', label='Equações recursivas');
plt.xlabel("t");
plt.ylabel("$\mathrm{x_{1}(t)}$");
plt.legend(loc='lower right');
plt.tight_layout();
plt.grid();
plt.show();

# Comportamento do estado x2:

plt.figure();
plt.plot(k*T, x2, ds='steps-post');
plt.plot(t, X1b, label='Simulação');
plt.plot(k*T, x2, 'r*', label='Equações recursivas');
plt.xlabel("t");
plt.ylabel("$\mathrm{x_{2}(t)}$");
plt.legend(loc='lower right');
plt.tight_layout();
plt.grid();
plt.show();

# Comportamento do estado x3:

plt.figure();
plt.plot(k*T, x3, ds='steps-post');
plt.plot(t, X1c, label='Simulação');
plt.plot(k*T, x3, 'r*', label='Equações recursivas');
plt.xlabel("t");
plt.ylabel("$\mathrm{x_{3}(t)}$");
plt.legend(loc='lower right');
plt.tight_layout();
plt.grid();
plt.show();

# Comportamento da saída y:

plt.figure();
plt.plot(k*T, y, ds='steps-post');
plt.plot(t, Y.T[0], label='Simulação');
plt.plot(k*T, y, 'r*', label='Equações recursivas');
plt.xlabel("t");
plt.ylabel("y(t)");
plt.legend(loc='lower right');
plt.tight_layout();
plt.grid();
plt.show();

# Comportamento da estado x1e:

plt.figure();
plt.plot(k*T, x1e, ds='steps-post');
plt.plot(t, Y.T[0], label='Simulação');
plt.plot(k*T, x1e, 'r*', label='Equações recursivas');
plt.xlabel("t");
plt.ylabel("$\mathrm{\\widetilde{x}_{1}(t)}$");
plt.legend(loc='lower right');
plt.tight_layout();
plt.grid();
plt.show();

# Comportamento da estado x2e:

plt.figure();
plt.plot(k*T, x2e, ds='steps-post');
plt.plot(t, Y.T[1], label='Simulação');
plt.plot(k*T, x2e, 'r*', label='Equações recursivas');
plt.xlabel("t");
plt.ylabel("$\mathrm{\\widetilde{x}_{2}(t)}$");
plt.legend(loc='lower right');
plt.tight_layout();
plt.grid();
plt.show();

# Comportamento da estado x3e:

plt.figure();
plt.plot(k*T, x3e, ds='steps-post');
plt.plot(t, Y.T[2], label='Simulação');
plt.plot(k*T, x3e, 'r*', label='Equações recursivas');
plt.xlabel("t");
plt.ylabel("$\mathrm{\\widetilde{x}_{3}(t)}$");
plt.legend(loc='lower right');
plt.tight_layout();
plt.grid();
plt.show();

## Controlador servossistema tipo 0 + observador de ordem plena

In [ ]:
tfinal = 0.1;
Kmax = int((tfinal/Ts)+1);
t =np.linspace(0, tfinal, Kmax);
r = np.ones(int(len(t)/2));
r = np.concatenate((r, 1.5*r), axis=0)
csi = np.zeros(len(t));
csi_ponto = np.zeros(len(t));
u = np.zeros(len(t));

# vetores de estados
x1_rec = np.zeros(len(t));
x2_rec = np.zeros(len(t));
x1_ponto_rec = np.zeros(len(t));
x2_ponto_rec = np.zeros(len(t));

# vetor de saída Y = C*X
y = np.zeros(len(t));

# vetores do observador de estados
til_x1_rec = np.zeros(len(t));
til_x2_rec = np.zeros(len(t));
til_x1_ponto_rec = np.zeros(len(t));
til_x2_ponto_rec = np.zeros(len(t));
til_y = np.zeros(len(t));


# Realimentação de controle do observador
erro_1 = np.zeros(len(t));
erro_2 = np.zeros(len(t));

## Condições iniciais

csi_ponto[0] = r[0] - y[0]; 

x1_ponto_rec[0] = A[0,0]*x1_rec[0] + A[0,1]*x2_rec[0] + B[0]*u[0];
x2_ponto_rec[0] = A[1,0]*x1_rec[0] + A[1,1]*x2_rec[0] + B[1]*u[0];
til_x1_ponto_rec[0]=B[0]*u[0];
til_x2_ponto_rec[0]=B[1]*u[0];

for j in range(0,Kmax-1):
    # sistema original
    csi[j] = Ts*csi_ponto[j-1] + csi[j-1];
    x1_rec[j] = Ts*x1_ponto_rec[j-1] + x1_rec[j-1];
    x2_rec[j] = Ts*x2_ponto_rec[j-1] + x2_rec[j-1];

     # observador 
    til_x1_rec[j] = Ts*til_x1_ponto_rec[j-1] + til_x1_rec[j-1]
    til_x2_rec[j] = Ts*til_x2_ponto_rec[j-1] + til_x2_rec[j-1]
    
    # Equação de Saída: Y=C*X
    y[j] = C[0,0]*x1_rec[j];


    # Equação de erro
    csi_ponto[j] = r[j] - y[j];     


    # Equação da lei de controle
    u[j] = -(K[0]*til_x1_rec[j] + K[1]*til_x2_rec[j]) + Ki*csi[j];


        # Equação de Saída do observador : tilY=C*tilX
    til_y[j] = C[0,0]*til_x1_rec[j] + C[0,1]*til_x2_rec[j];

    #Equações da lei de controle
    erro_1[j] = Ke[0]*(y[j] - til_y[j]);
    erro_2[j] = Ke[1]*(y[j] - til_y[j]);


    # Equação diferencial de estados de x_ponto e til_x_ponto
    x1_ponto_rec[j] = A[0,0]*x1_rec[j] + A[0,1]*x2_rec[j] + B[0]*u[j];
    x2_ponto_rec[j] = A[1,0]*x1_rec[j] + A[1,1]*x2_rec[j] + B[1]*u[j];
    til_x1_ponto_rec[j] = A[0,0]*til_x1_rec[j] + A[0,1]*til_x2_rec[j] + B[0]*u[j] + erro_1[j];
    til_x2_ponto_rec[j] = A[1,0]*til_x1_rec[j] + A[1,1]*til_x2_rec[j] + B[1]*u[j] + erro_2[j];


fig, ((ax1, ax2) ) = plt.subplots(2, 1)
fig.set_size_inches(20,8)

ax1.plot(t, x1_rec, t, til_x1_rec,'*')
ax1.set_title("x1 e x1 do observador utilizando recursivas")
ax1.set_xlabel("t")
ax1.set_ylabel("x1(t)")
ax1.legend(['X1 recursiva','X~1 recursiva'],loc="lower right")
ax1.grid(True)

ax2.plot(t, x2_rec, t, til_x2_rec,'*')
ax2.set_title("x2 e x2 do observador utilizando recursivas")
ax2.set_xlabel("t")
ax2.set_ylabel("x2(t)")
ax2.legend(['X2 recursiva','X~2 recursiva'],loc="lower right")
ax2.grid(True)

## Controlador servossistema tipo 1 + observador de ordem plena

In [ ]:
K = np.matrix('25 5')
Ke = np.matrix('198; 9604')

k1 = K[0, 0]

# simulação com discretização dos integradores
# I(z)=T/(z-1)=Xi(z)/Xi_ponto(z)
# xi(k)=T*xi_ponto(k-1)+xi(k-1)

T = 0.01;
tfinal = 5;
Kmax = int(tfinal/T+1);
k = np.linspace(0, Kmax, Kmax+1)
ref = np.ones(len(k));

x1 = np.zeros(len(k))
x2 = np.zeros(len(k))

x1_ponto = np.zeros(len(k))
x2_ponto = np.zeros(len(k))

xtil1 = np.zeros(len(k))
xtil2 = np.zeros(len(k))

xtil1_ponto = np.zeros(len(k))
xtil2_ponto = np.zeros(len(k))

y = C[0,0]*x1

ytil = C[0,0]*x1_til

erro = np.zeros(len(k))

u = np.zeros(len(k))
    for j in range(1,Kmax+1):
    # sistema original
        # Equações dos integradores
        x1[j]=T*x1_ponto[j-1]+x1[j-1];
        x2[j]=T*x2_ponto[j-1]+x2[j-1];    

        xtil1[j]=T*xtil1_ponto[j-1]+xtil1[j-1]; 
        xtil2[j]=T*xtil2_ponto[j-1]+xtil2[j-1]; 

        # equação de saída do observador de ordem mínima
        # Equação de Saída: Y=C*X+D*U
        y[j] = C[0,0]*x1[j]
        ytil[j] = C[0,0]*xtil1[j]
        
        erro[j] = ref[j] - xtil1[j];
        
        u[j] = -(K[0,1]*xtil2[j]) + (k1*(erro[j]));
        # Equação diferencial de estados:  Xponto=A*X+B*U
        x1_ponto[j] = A[0,0]*x1[j]+A[0,1]*x2[j]+B[0]*u[j];  
        x2_ponto[j] = A[1,0]*x1[j]+A[1,1]*x2[j]+B[1]*u[j]; 

        
        xtil1_ponto[j] = A[0,0]*xtil1[j]+A[0,1]*xtil2[j]+B[0]*u[j] + Ke[0]*(y[j] - ytil[j]);  
        xtil2_ponto[j] = A[1,0]*xtil1[j]+A[1,1]*xtil2[j]+B[1]*u[j] + Ke[1]*(y[j] - ytil[j]); 
        

## Controlador servossitema tipo 0 + observador de ordem mínima

In [ ]:
T = 1/Fs;
tfinal = 0.05;
Kmax = int(tfinal/T+1);
k = np.linspace(0, Kmax, int(Fs*tfinal))


# definindo os vetores
r = np.ones(len(k));
u = np.zeros(len(k));
x1 = np.zeros(len(k));
x2 = np.zeros(len(k));
csi = np.zeros(len(k));
x1_ponto = np.zeros(len(k));
x2_ponto = np.zeros(len(k));
csi_ponto = np.zeros(len(k));
y = np.zeros(len(k));
eta1 = np.zeros(len(k));
eta1_ponto = np.zeros(len(k));
xtil1 = np.zeros(len(k));
xtil2 = np.zeros(len(k));

# condições iniciais (podem ser diferentes de zero)
x1[0] = 0;  #  para k = 0
x2[0] = 0;  #  para k = 0
x1_ponto[0] = 0;  #  para k = 0
x2_ponto[0] = 0;  #  para k = 0
y[0] = C[0,0]*x1[0]+C[0,1]*x2[0]+D*u[0];  #  para k = 0
eta1[0] = 0;  #  para k = 0
eta1_ponto[0] = Fchapeu*u[0];  #  para k = 0
xtil1[1]=Cchapeu[0,0]*eta1[0]+Dchapeu[0]*y[0];
xtil2[1]=Cchapeu[1,0]*eta1[0]+Dchapeu[1]*y[0];
csi_ponto[0] = r[0] - y[0];



for j in range(1,Kmax-1):
    # sistema original
        # Equações dos integradores
        x1[j]=T*x1_ponto[j-1]+x1[j-1];
        x2[j]=T*x2_ponto[j-1]+x2[j-1];
        csi[j] = Ts*csi_ponto[j-1] + csi[j-1];
    
        # Equação da lei de controle + observador
        u[j] = -(K[0]*xtil1[j] + K[1]*xtil2[j]) + Ki*csi[j];
        
        # Equação diferencial de estados:  Xponto=A*X+B*U
        x1_ponto[j]=A[0,0]*x1[j]+A[0,1]*x2[j]+B[0]*u[j];  
        x2_ponto[j]=A[1,0]*x1[j]+A[1,1]*x2[j]+B[1]*u[j]; 
        
    # observador de ordem mínima
        # Equações dos integradores do observador
        eta1[j]=T*eta1_ponto[j-1]+eta1[j-1];
    
        
        # Equação de Saída: Y=C*X+D*U
        y[j] = C[0,0]*x1[j]+C[0,1]*x2[j]+D*u[j];
        
        # Equação diferencial de estados do observador
        eta1_ponto[j]=Achapeu*eta1[j]+Bchapeu*y[j]+Fchapeu*u[j];
        

        
        if j != (Kmax -2):
     # Transformação
        # equação de saída do observador de ordem mínima
            xtil1[j+1]=Cchapeu[0,0]*eta1[j]+Dchapeu[0]*y[j];
            xtil2[j+1]=Cchapeu[1,0]*eta1[j]+Dchapeu[1]*y[j];
        
        # Equação de erro
        csi_ponto[j] = r[j] - y[j];

## Controlador servossistema tipo 1 + observador de ordem minima

In [ ]:
T = 1/Fs;
tfinal = 0.05;
Kmax = int(tfinal/T+1);
k = np.linspace(0, Kmax, int(Fs*tfinal))


# definindo os vetores
r = np.ones(len(k));
u = np.zeros(len(k));
x1 = np.zeros(len(k));
x2 = np.zeros(len(k));
x1_ponto = np.zeros(len(k));
x2_ponto = np.zeros(len(k));
erro = np.zeros(len(k));
y = np.zeros(len(k));
eta1 = np.zeros(len(k));
eta1_ponto = np.zeros(len(k));
xtil1 = np.zeros(len(k));
xtil2 = np.zeros(len(k));

# condições iniciais (podem ser diferentes de zero)
x1[0] = 0;  #  para k = 0
x2[0] = 0;  #  para k = 0
x1_ponto[0] = 0;  #  para k = 0
x2_ponto[0] = 0;  #  para k = 0
y[0] = C[0,0]*x1[0]+C[0,1]*x2[0]+D*u[0];  #  para k = 0
eta1[0] = 0;  #  para k = 0
eta1_ponto[0] = Fchapeu*u[0];  #  para k = 0
xtil1[1]=Cchapeu[0,0]*eta1[0]+Dchapeu[0]*y[0];
xtil2[1]=Cchapeu[1,0]*eta1[0]+Dchapeu[1]*y[0];
erro[0] = r[0] - y[0];



for j in range(1,Kmax-1):
    # sistema original
        # Equações dos integradores
        x1[j]=T*x1_ponto[j-1]+x1[j-1];
        x2[j]=T*x2_ponto[j-1]+x2[j-1];
    
        # Equação da lei de controle + observador
        u[j] = -(K[1]*xtil2[j]) + K[0]*erro[j];
        
        # Equação diferencial de estados:  Xponto=A*X+B*U
        x1_ponto[j]=A[0,0]*x1[j]+A[0,1]*x2[j]+B[0]*u[j];  
        x2_ponto[j]=A[1,0]*x1[j]+A[1,1]*x2[j]+B[1]*u[j]; 
        
    # observador de ordem mínima
        # Equações dos integradores do observador
        eta1[j]=T*eta1_ponto[j-1]+eta1[j-1];
    
        
        # Equação de Saída: Y=C*X+D*U
        y[j] = C[0,0]*x1[j]+C[0,1]*x2[j]+D*u[j];
        
        # Equação diferencial de estados do observador
        eta1_ponto[j]=Achapeu*eta1[j]+Bchapeu*y[j]+Fchapeu*u[j];
        

        
        if j != (Kmax -2):
     # Transformação
        # equação de saída do observador de ordem mínima
            xtil1[j+1]=Cchapeu[0,0]*eta1[j]+Dchapeu[0]*y[j];
            xtil2[j+1]=Cchapeu[1,0]*eta1[j]+Dchapeu[1]*y[j];
        
        # Equação de erro
        erro[j] = r[j] - y[j];